In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df = pd.read_csv(r'C:\Users\dream\OneDrive\Desktop\Flight_Fare_Prediction\DATA\new_df.csv')

In [3]:
df.head()

,Journey_day,Airline,Class,Source,Total_stops,Destination,Duration_in_hours,Days_left,Fare,On_weekend,Daytime_departure,Daytime_arrival
0,16,Other,Economy,Delhi,non-stop,Mumbai,2.0833,1,5335,False,False,False
1,16,Indigo,Economy,Delhi,non-stop,Mumbai,2.3333,1,5899,False,False,False
2,16,GO FIRST,Economy,Delhi,non-stop,Mumbai,2.1667,1,5801,False,False,False
3,16,Other,Economy,Delhi,non-stop,Mumbai,2.0833,1,5794,False,False,False
4,16,Air India,Economy,Delhi,non-stop,Mumbai,2.1667,1,5955,False,False,False


In [4]:
# Convert boolean columns to integers
df['On_weekend'] = df['On_weekend'].astype(int)
df['Daytime_departure'] = df['Daytime_departure'].astype(int)
df['Daytime_arrival'] = df['Daytime_arrival'].astype(int)

# One-hot encode categorical variables
df_encoded = pd.get_dummies(df, columns=['Airline', 'Class', 'Source', 'Total_stops', 'Destination'])

# Define features and target
X = df_encoded.drop(columns=['Fare'])
y = df_encoded['Fare']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = np.array(X_train).astype(np.float32)
X_test = np.array(X_test).astype(np.float32)
y_train = np.array(y_train).astype(np.float32)
y_test = np.array(y_test).astype(np.float32)

In [5]:
df_encoded.head()

,Journey_day,Duration_in_hours,Days_left,Fare,On_weekend,Daytime_departure,Daytime_arrival,Airline_Air India,Airline_AirAsia,Airline_GO FIRST,...,Total_stops_1-stop,Total_stops_2+-stop,Total_stops_non-stop,Destination_Ahmedabad,Destination_Bangalore,Destination_Chennai,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_Mumbai
0,16,2.0833,1,5335,0,0,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,16,2.3333,1,5899,0,0,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,16,2.1667,1,5801,0,0,0,False,False,True,...,False,False,True,False,False,False,False,False,False,True
3,16,2.0833,1,5794,0,0,0,False,False,False,...,False,False,True,False,False,False,False,False,False,True
4,16,2.1667,1,5955,0,0,0,True,False,False,...,False,False,True,False,False,False,False,False,False,True


# **NEURAL NETWORKS**


In [6]:
# Define the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Mean Squared Error on test set: {loss}')

# Predict and compare
y_pred = model.predict(X_test)
print(y_pred[:10])  # Display the first 10 predictions

Epoch 1/50


c:\Users\dream\OneDrive\Desktop\Flight_Fare_Prediction\.conda\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7912/7912 ━━━━━━━━━━━━━━━━━━━━ 8s 931us/step - loss: 306372096.0000 - val_loss: 48492572.0000
Epoch 2/50
7912/7912 ━━━━━━━━━━━━━━━━━━━━ 8s 994us/step - loss: 47167204.0000 - val_loss: 45173392.0000
Epoch 3/50
7912/7912 ━━━━━━━━━━━━━━━━━━━━ 7s 898us/step - loss: 44723836.0000 - val_loss: 44257152.0000
Epoch 4/50
7912/7912 ━━━━━━━━━━━━━━━━━━━━ 7s 888us/step - loss: 44082148.0000 - val_loss: 43876080.0000
Epoch 5/50
 247/7912 ━━━━━━━━━━━━━━━━━━━━ 6s 819us/step - loss: 45993496.0000

KeyboardInterrupt: 

# **ALL OTHER REGRESSION MODEL**

In [7]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt

In [8]:
# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200, 500]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='roc_auc', n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and the best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Evaluate the best model
y_pred_best = best_model.predict(X_test)
y_prob_best = best_model.predict_proba(X_test)[:, 1]
fpr_best, tpr_best, thresholds_best = roc_curve(y_test, y_prob_best)
auc_score_best = roc_auc_score(y_test, y_prob_best)

# Plot the ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr_best, tpr_best, label=f'ROC Curve (AUC = {auc_score_best:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

# Print evaluation metrics
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
print(f'Best Parameters: {best_params}')
print(f'Accuracy: {accuracy_best:.2f}')
print(f'Precision: {precision_best:.2f}')
print(f'Recall: {recall_best:.2f}')

c:\Users\dream\OneDrive\Desktop\Flight_Fare_Prediction\.conda\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt

In [16]:
# Define the parameter grids
param_grids = {
    # 'DecisionTreeClassifier': {
    #     'max_depth': [None, 10, 20, 30],
    #     'min_samples_split': [2, 5, 10],
    #     'min_samples_leaf': [1, 2, 4]
    # },
    'RandomForestClassifier': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    # 'SVC': {
    #     'C': [0.1, 1, 10],
    #     'kernel': ['linear', 'rbf', 'poly'],
    #     'probability': [True]
    # },
    # 'GradientBoostingClassifier': {
    #     'n_estimators': [50, 100, 200],
    #     'learning_rate': [0.01, 0.1, 0.2],
    #     'max_depth': [3, 5, 7]
    # },
    # 'KNeighborsClassifier': {
    #     'n_neighbors': [3, 5, 7],
    #     'weights': ['uniform', 'distance']
    }

# Function to perform hyperparameter tuning and plot ROC curve
def evaluate_model(model, param_grid, X_train, y_train, X_test, y_test, model_name):
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    y_pred = best_model.predict(X_test)
    y_prob = best_model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    auc_score = roc_auc_score(y_test, y_prob)

    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_score:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'{model_name} ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    print(f'Best Parameters for {model_name}: {best_params}')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')

    return best_params

# Storage for best parameters
best_params_dict = {}

# Evaluate each model
models = {
    # 'DecisionTreeClassifier': DecisionTreeClassifier(),
    'RandomForestClassifier': RandomForestClassifier()
    # 'SVC': SVC(),
    # 'GradientBoostingClassifier': GradientBoostingClassifier(),
    # 'KNeighborsClassifier': KNeighborsClassifier()
}

for model_name, model in models.items():
    print(f'Evaluating {model_name}...')
    best_params = evaluate_model(model, param_grids[model_name], X_train, y_train, X_test, y_test, model_name)
    best_params_dict[model_name] = best_params

# Print the dictionary of best parameters
print("Best Parameters for each model:")
print(best_params_dict)


Evaluating RandomForestClassifier...


c:\Users\dream\anaconda3\envs\ffvenv\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: 
All the 180 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\dream\anaconda3\envs\ffvenv\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\dream\anaconda3\envs\ffvenv\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\dream\anaconda3\envs\ffvenv\Lib\site-packages\sklearn\ensemble\_forest.py", line 489, in fit
    trees = Parallel(
            ^^^^^^^^^
  File "c:\Users\dream\anaconda3\envs\ffvenv\Lib\site-packages\sklearn\utils\parallel.py", line 74, in __cal